In [1]:
import requests
from bs4 import BeautifulSoup

In [28]:
URL ='https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/'  #põe as chaves no final para concatenar depois

In [29]:
c = requests.get(URL).content
soup = BeautifulSoup(c)

In [1]:
#soup

In [31]:
#você pega a classe, pelo código fonte HTML, da parte que você quer a informação. Utilizando o "inspecionar elemento", analisando o código fonte da página. 
# No caso, ela quer o texto do anúncio.

nome_anuncio = soup.find_all('span', {'class':'property-card__title js-cardLink js-card-title'})

In [45]:
nome_anuncio[0].text

'  Apartamento com  Quarto para Aluguel, 40m²  '

In [19]:
#quando muda de página, o URL muda; só o final dele.
#exemplo: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=2

paginas = [''] #na primeira página não tem nada, então deixa vazio
for i in range(2, 5):
    a = '?pagina='+str(i)
    paginas.append(a)

In [20]:
paginas

['', '?pagina=2', '?pagina=3', '?pagina=4']

In [21]:
import pandas as pd
def converter_df(aluguel_lista, quartos_lista, endereco_lista, banheiro_lista, vaga_lista):
    data = {'aluguel': aluguel_lista, 'quartos': quartos_lista, 'endereco': endereco_lista, 'banheiro': banheiro_lista,'vaga': vaga_lista}
    df = pd.DataFrame(data)
    return df

In [59]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    
    aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'})
    aluguel_lista = [p.text for p in aluguel_tag]
    
    quartos_tag = soup.find_all('li', {'class':'property-card__detail-item property-card__detail-room js-property-detail-rooms'})
    quartos_lista = [en.text for en in quartos_tag]

    endereco_tag = soup.find_all('span', {'class':'property-card__address'})
    endereco_lista = [fg.text for fg in endereco_tag]
    
    banheiro_tag = soup.find_all('li', {'class':'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'})
    banheiro_lista = [yp.text for yp in banheiro_tag]
    
    vaga_tag = soup.find_all('li', {'class':'property-card__detail-item property-card__detail-garage js-property-detail-garages'})
    vaga_lista = [vg.text for vg in vaga_tag]
    
    df = converter_df(aluguel_lista, quartos_lista, endereco_lista, banheiro_lista, vaga_lista)
    
    return df

In [68]:
import time

URL2 ='https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/{}'  #põe as chaves no final para concatenar depois
df_lista = []
for pg in paginas:
    url = URL2.format(pg)
    print('Coletando URL:', url)
    try:
        df = scrap(url)
    except Exception as e: #adiciona uma exceção. O 'e' é de erro.
        print('Erro ao coletar dados na url:', url)
        print(e)
    df_lista.append(df)
    time.sleep(5)
    
df_final = pd.concat(df_lista)

Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=2
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=3
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=4


In [61]:
df_final.head()

,aluguel,quartos,endereco,banheiro,vaga
0,R$ 1.800 /Mês,1 Quarto,"Rua Juquis, 31 - Indianópolis, São Paulo - SP",1 Banheiro,-- Vaga
1,R$ 4.000 /Mês,2 Quartos,"Rua Major Quedinho, 224 - Centro, São Paulo - SP",2 Banheiros,1 Vaga
2,R$ 6.850 /Mês,3 Quartos,"Avenida Nazaré, 1180 - Ipiranga, São Paulo - SP",5 Banheiros,2 Vagas
3,R$ 1.800 /Mês,1 Quarto,"Rua André Casado, 215 - Sumaré, São Paulo - SP",1 Banheiro,1 Vaga
4,R$ 5.165 /Mês R$ 1.289.900,3 Quartos,"Rua Helion Póvoa, 155 - Vila Olímpia, São Paul...",2 Banheiros,2 Vagas


In [63]:
df_final['bairro'] = df_final.endereco.str.split('\s-').str[1]  #seleciona pelo regex (expressão regular), depois do espaço e hífen
df_final['rua'] = df_final.endereco.str.split('\s-').str[0] #seleciona pelo regex (expressão regular), antes do espaço e hífen

In [66]:
df_final.head()

,aluguel,quartos,endereco,banheiro,vaga,bairro,rua
0,R$ 1.800 /Mês,1 Quarto,"Rua Juquis, 31 - Indianópolis, São Paulo - SP",1 Banheiro,-- Vaga,"Indianópolis, São Paulo","Rua Juquis, 31"
1,R$ 4.000 /Mês,2 Quartos,"Rua Major Quedinho, 224 - Centro, São Paulo - SP",2 Banheiros,1 Vaga,"Centro, São Paulo","Rua Major Quedinho, 224"
2,R$ 6.850 /Mês,3 Quartos,"Avenida Nazaré, 1180 - Ipiranga, São Paulo - SP",5 Banheiros,2 Vagas,"Ipiranga, São Paulo","Avenida Nazaré, 1180"
3,R$ 1.800 /Mês,1 Quarto,"Rua André Casado, 215 - Sumaré, São Paulo - SP",1 Banheiro,1 Vaga,"Sumaré, São Paulo","Rua André Casado, 215"
4,R$ 5.165 /Mês R$ 1.289.900,3 Quartos,"Rua Helion Póvoa, 155 - Vila Olímpia, São Paul...",2 Banheiros,2 Vagas,"Vila Olímpia, São Paulo","Rua Helion Póvoa, 155"
